In [1]:
import numpy as np
import sys
import hrr_common.utils as cu
import hrr_common as hrr_cm
import hrr_cobot_robot
import spatialmath as sm
import rospy
from tqdm.notebook import tqdm, trange

# ROS
import actionlib
np.set_printoptions(suppress=True)

In [2]:
def get_hostname_and_IP():
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.connect(("8.8.8.8", 80))
        return socket.gethostname(), s.getsockname()[0]

_host, _ip = get_hostname_and_IP()
%env ROS_MASTER_URI=http://hrrcobotLinux54.lsr.ei.tum.de:11311
# %env ROS_MASTER_URI=http://localhost:11311
print(f"\t\033[1m\033[4m!!!please check that the IPs below is identical!!!\033[0m"+
      f"\n\thost {_host} has IP {_ip}, which should be identical to",end="\nvs:\t")
%env ROS_IP=$_ip

for a local setup, replace the above by:
	%env ROS_MASTER_URI=http://localhost:11311
env: ROS_MASTER_URI=http://hrrN3511rt2004.lsr.ei.tum.de:11311
in case this notebook does not work please check that IP below is identical:
	%env ROS_IP=129.187.147.174		<=>	env: ROS_IP=129.187.147.174


In [3]:
np.set_printoptions(suppress=True)

## Launch

catkin build hrr_ee_tools package

Launch the cobot
```bash
roslaunch hrr_cobot_robot hrr_cobot_hw.launch tool_name:="vacuum"  
```

```bash
rosrun planner planner_server_v2.py 
```
```bash
rosrun hrr_cobot_robot enable_sns_trk --sns_controller_name sensor_track_velocity_controller
```

In [4]:
rospy.init_node('vacuum_grasping_dev')


[ERROR] [1641980871.027711]: could not set tool-frame @ /vacuum_grasping_dev due to missing ROS-parameter: '~tool_name'


[INFO] [1641980871.511826]: Updated force  gains: K_p:=[0.0001, 0.0001, 0.0001]
[INFO] [1641980871.515564]: Updated torque gains: K_p:=[0.0005, 0.0005, 0.0005]
q:		[   4.09713636    4.22352218 -105.27729153  -88.65102648   93.92755252
  -25.46458276][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.45649518 0.04729539 0.65235046][m]
quat:		-0.27107 <[0.27024388 0.65315031 0.6533643 ]>
tool-pos:	[0.45649518 0.04729539 0.65235046][m]
tool-rpy:	[-134.98979187   -0.05507659  -89.95841217][°]
robot-status:	terminate


[WARN] [1641980871.749308]: Could not process inbound connection: Client [/hrr_cobot] wants topic [/hrr_cobot/sensor_track_compliant_controller/hybrid_ctrl_cmd] to have datatype/md5sum [hrr_msgs/HybridForceVelocityCmdStamped/6bfe9d92f9569269ce7086e736ecd0c2], but our version has [hrr_msgs/HybridForceVelocityCmdStamped/c7d32c882fe005053aba472f934842bd] Dropping connection.{'callerid': '/hrr_cobot', 'md5sum': '6bfe9d92f9569269ce7086e736ecd0c2', 'tcp_nodelay': '0', 'topic': '/hrr_cobot/sensor_track_compliant_controller/hybrid_ctrl_cmd', 'type': 'hrr_msgs/HybridForceVelocityCmdStamped'}


In [15]:
hrr_cobot_robot.utils.load_default_parameters("/hrr_cobot")
cobot = hrr_cobot_robot.HrrCobotControl.from_ros("/hrr_cobot")

NameError: name 'hrr_cobot_robot' is not defined

In [ ]:
print(cobot)

### Force tests

In [ ]:
%matplotlib widget
cobot.run_calibration_routine(file_name='/tmp/current_tool.npy', plot=True, figsize=(12,10))

In [ ]:
print(ft_status.wrench)

In [ ]:
print(cobot)

In [ ]:
cobot.init_sns_vel()

In [ ]:
cobot.reset()
cobot.set_compliant(np.r_[1.0, 1.0, 1.0, np.zeros(3)], Path("/tmp/current_tool.npy"), F_max=5.0)

In [ ]:
cobot.acknowledge_calibration()

In [ ]:
T = int(10 * cobot.hz)
for t in trange(T):
    cobot.update()

In [ ]:
cobot.reset()
cobot.set_py_hybrid_force_vel_command(B_F_des=np.r_[0.0, 0.0, -10.0, np.zeros(3)], 
                                          K_f=8e-4, K_t=5e-3, scale_pos=0.0, scale_rot=0.0, 
                                      wrench_dir=np.r_[1.0, 1.0, 1.0, np.ones(3)],
                                      vel_dir=np.zeros(6))

### Vacuum tests


In [ ]:
#Usage fo
for t in trange(20):
    tool_controller.vacuum = True
    rospy.sleep(0.2)

### Use cases
Emergency lamp: 
    Remove and dispose lid after it is cut loose from the fixtures
    Remove and dispose middle cover
Microwave Oven: None
Pc Tower: 
    Dispose PSU after it is pushed from the housing
    Dipose CPU cooler after it is unscrewed


## Params from outside

In [2]:
import hrr_common.utils as cu
import hrr_common.ros_utils as ru
def getSE3fromPoseStamped(msg):
        p = np.array(
            [msg.pose.position.x, msg.pose.position.y, msg.pose.position.z])
        q = ru.quat2np(msg.pose.orientation)
        return cu.posquat2homog(p, q)

In [3]:
from geometry_msgs.msg import PoseStamped #For testing purposes
object_center = PoseStamped() #PoseStamped
release_pose = PoseStamped() #PoseStamped

In [4]:
object_center.pose.position.x = 0.66
object_center.pose.position.y = 0.17
object_center.pose.position.z = 0.07
object_center.pose.orientation.x = 0
object_center.pose.orientation.y = -0.715468876127
object_center.pose.orientation.z = 0.0542776712358
object_center.pose.orientation.w = 0.696533001155

In [5]:
obj_cent = getSE3fromPoseStamped(object_center)
obj_cent

  -0.02968  -0.07561  -0.9967    0.66      
   0.07561   0.9941   -0.07767   0.17      
   0.9967   -0.07767  -0.02379   0.07      
   0         0         0         1         


In [8]:
surface_nvec = obj_cent.A[:3,0]
approach_vec = -surface_nvec
surface_nvec

array([-0.02968356,  0.07561238,  0.99669537])

In [14]:
robot_pose = cu.calc_goal_pose(surface_nvec, obj_cent.t, y_axis=obj_cent.A[:3,1])
robot_pose

  -0.9967   -0.07561   0.02968   0.66      
  -0.07767   0.9941   -0.07561   0.17      
  -0.02379  -0.07767  -0.9967    0.07      
   0         0         0         1         


In [16]:
robot_pose @ sm.SE3([0,0,-0.1]) #tool tip at robot_pose

  -0.9967   -0.07561   0.02968   0.657     
  -0.07767   0.9941   -0.07561   0.1776    
  -0.02379  -0.07767  -0.9967    0.1697    
   0         0         0         1         


In [19]:
posediff = robot_pose @ sm.SE3([0,0,1]) - robot_pose

In [23]:
posediff[:3,-1], -surface_nvec

(array([ 0.02968356, -0.07561238, -0.99669537]),
 array([ 0.02968356, -0.07561238, -0.99669537]))

In [56]:
sm.SE3.Ry(np.deg2rad(90))

   0         0         1         0         
   0         1         0         0         
  -1         0         0         0         
   0         0         0         1         


In [19]:
jointys = np.deg2rad([-22.855, 58.494,-115.143,-22.934,-85.570,-267.116])

In [40]:
from sim_robots import get_racer_kin_model
kin_model = get_racer_kin_model()
kin_model.forward_kin(jointys), ru.rotmat2quat(kin_model.forward_kin(jointys).A)

(  -0.0004249  0.02499   0.9997    0.5019    
  -1        -0.007128 -0.0002468  0.1778    
   0.00712  -0.9997    0.025     0.1823    
   0         0         0         1         
,
 x: -0.49540562250119097
 y: 0.49201115754681823
 z: -0.5080722978241229
 w: 0.5043419774591079)

In [41]:
qcalib = np.r_[0., 0., -np.pi / 2.0, 0., np.pi / 2.0, 0.0]
kin_model.forward_kin(qcalib), ru.rotmat2quat(kin_model.forward_kin(qcalib).A)

(  -1         0         0         0.4359    
   0         1         0         0         
   0         0        -1         0.705     
   0         0         0         1         
,
 x: -0.0
 y: -1.0
 z: -0.0
 w: 0.0)

In [42]:
qcalib2 = np.r_[4, 0., -np.pi / 2.0, 0., np.pi / 2.0, 3.0]
kin_model.forward_kin(qcalib2), ru.rotmat2quat(kin_model.forward_kin(qcalib2).A)

(  -0.5403    0.8415    0        -0.2849    
   0.8415    0.5403    0         0.3299    
   0         0        -1         0.705     
   0         0         0         1         
,
 x: 0.479425538604203
 y: 0.8775825618903726
 z: 2.1479277849233506e-17
 w: -5.734143423154037e-17)

In [43]:
q0 = np.r_[0,0,-np.pi/2,0,0,0]
kin_model.forward_kin(q0), ru.rotmat2quat(kin_model.forward_kin(q0).A)

(   0         0         1         0.5159    
   0         1         0         0         
  -1         0         0         0.785     
   0         0         0         1         
,
 x: -0.0
 y: 0.7071067811865475
 z: -0.0
 w: 0.7071067811865475)

In [ ]:
cu.rotmat2quat()

In [ ]:
cu.quat2human(object_center.pose.orientation)

In [ ]:
#Gerade noch oben bedeutet Rotmatrix = np.eye(3)

In [21]:
pos_objC = np.array([object_center.pose.position.x, object_center.pose.position.y,object_center.pose.position.z])
quat_objC = np.quaternion()
quat_objC.x = object_center.pose.orientation.x
quat_objC.y = object_center.pose.orientation.y
quat_objC.z = object_center.pose.orientation.z
quat_objC.w = object_center.pose.orientation.w
obj_center = cu.posquat2homog(pos_objC, quat_objC)

In [22]:
pos_rp = np.array([release_pose.pose.position.x, release_pose.pose.position.y,release_pose.pose.position.z])
quat_rp = np.quaternion()
quat_rp.x = release_pose.pose.orientation.x
quat_rp.y = release_pose.pose.orientation.y
quat_rp.z = release_pose.pose.orientation.z
quat_rp.w = release_pose.pose.orientation.w
rel_pose = cu.posquat2homog(pos_rp, quat_rp)

   1         0         0         0         
   0         1         0         0         
   0         0         1         0         
   0         0         0         1         


# Stuff to test in lab

In [ ]:
cobot.quat

In [ ]:
cobot.T_B_E_robot

In [ ]:
cobot.q

In [ ]:
#Test above quaternion stuff and how it relates to SE3 to understand
#--> Probably should ask for normal vector instead of pose if CERTH doesnt know what they are doing
#1. What does quaternion [0 0 0 0] mean for our case?

#2. 

In [ ]:
#Go to position with Robot EE while pointing straight down with vacuum gripper


In [ ]:
#Turn the last link to rotate vacuum gripper


In [ ]:
#Be compliant in base direction 4,5 while going straight down (calibrate first), can we use this for vacuum grasping?


### What we really need: 
* pose of the robot when activating vacuum gripper (call it goal_pose)
* pose of the robot when deactivating (call it dispose_pose)
* (maybe) a pre-pose or normal vector to the surface

### Load it manually for now

In [61]:
#cobot.update_tf()
#np.save('goal_pose_vac',cobot.T_B_E_robot.A,allow_pickle=True,) #saves current pose as goal_pose (for metal sheet of PC)
#np.save('goal_pose_vac_PSU',cobot.T_B_E_robot.A,allow_pickle=True,) #saves current pose as goal_pose
goal_pose = np.load('goal_pose_vac_PSU.npy')
sm.SE3(goal_pose)

  -0.712    -0.7021    0.01307   0.6115    
  -0.7022    0.7119   -0.00851  -0.3118    
  -0.003332 -0.01524  -0.9999    0.31      
   0         0         0         1         


In [118]:
#np.save('dispose_pose_vac',dispose_pose.A,allow_pickle=True) #saves current pose as release_pose
dispose_pose = np.load('dispose_pose_vac.npy')
dispose_pose = sm.SE3(np.array([0,0,0.07]))@sm.SE3(dispose_pose) #for PSU go a bit higher with the dispose pose
sm.SE3(dispose_pose) #prob go down 10-20cm

  -0.6713   -0.7408    0.02192   0.5709    
  -0.7408    0.6716    0.01315   0.3754    
  -0.02447  -0.007406 -0.9997    0.505     
   0         0         0         1         


In [117]:
#np.save('final_pose_vac',final_pose.A,allow_pickle=True)
final_pose = np.load('final_pose_vac.npy')
final_pose = sm.SE3(np.array([0,0,0.07]))@sm.SE3(final_pose)
sm.SE3(final_pose)

  -0.6713   -0.7408    0.02192   0.5709    
  -0.7408    0.6716    0.01315   0.3754    
  -0.02447  -0.007406 -0.9997    0.355     
   0         0         0         1         


In [70]:
#These are used to compute the pre-pose:
nvec = np.array([0, 0, 1])
nvec = cu.normalize_vec(nvec) #Make sure nvec is normalized
pre_pose_dist = 0.15 #in meter 0.04 for metal sheet, 0.15

In [71]:
pre_pose = sm.SE3(nvec* pre_pose_dist) @ sm.SE3(goal_pose)
sm.SE3(pre_pose)

  -0.712    -0.7021    0.01307   0.6115    
  -0.7022    0.7119   -0.00851  -0.3118    
  -0.003332 -0.01524  -0.9999    0.46      
   0         0         0         1         


In [66]:
cobot.update_tf()
cobot.T_B_E_robot

  -0.712    -0.7021    0.01308   0.6115    
  -0.7022    0.7119   -0.008514 -0.3118    
  -0.003333 -0.01525  -0.9999    0.31      
   0         0         0         1         


In [67]:
print(cobot)

Hrr-Cobot state:
FT-data:
current force:	[-10.04425049   8.58032227   4.32421875] [N]
current torque:	[-9.0826416  -5.20385742  0.6362915 ] [Nm]
current wrench:	[-10.04794522   8.59073303   4.27519161  -9.11981641  -5.19068244   0.63117795] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[ 27.00190296  53.65089687 -73.68950394   0.12023661  51.77059702  71.53010548][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[ 0.61152948 -0.31176337  0.30997083][m]
quat:		0.00443 <[-0.3794597   0.92517536 -0.00642034]>
tool-pos:	[ 0.61152948 -0.31176337  0.30997083][m]
tool-rpy:	[135.40132141   0.74934298 179.51211548][°]
robot-status:	moving
---


### Initiate planner

In [19]:
 # Initialize the planner action client.
    
# >> Remeber to run the planner node (rosrun planner planner_server_v2.py) in terminal
# >> 
#rospy.init_node('planner_client')

client1 = actionlib.SimpleActionClient('planner', PlannerAction)
client1.wait_for_server()

# Sending deride end configuration
goal = PlannerGoal()
goal.goal_pose.header.frame_id = "map"
goal.goal_pose.header.stamp = rospy.Time.now()

## If we want to, we can start at a pose

In [132]:
cobot.move_to_pose( cobot.FK(cobot.q_calib), err_gain = None, v_max = 0.03) #start_pose, cobot.FK(cobot.q_calib)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1641983086.760093]: reached goal pose at step 712 / 10000


## Go-to pre-pose with traj planner
Assuming pre-pose is a translation of pre_pose_dist in the direction of nvec from goal_pose

In [133]:
pre_pose_quat = cu.homog2quat(pre_pose)
pre_pose_pos = cu.homog2pos(pre_pose)
#position of end_configuration
goal.goal_pose.pose.position.x = pre_pose_pos[0]
goal.goal_pose.pose.position.y = pre_pose_pos[1]
goal.goal_pose.pose.position.z = pre_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = pre_pose_quat.x
goal.goal_pose.pose.orientation.y = pre_pose_quat.y
goal.goal_pose.pose.orientation.z = pre_pose_quat.z
goal.goal_pose.pose.orientation.w = pre_pose_quat.w

In [134]:
client1.send_goal(goal)


[WARN] [1641983232.063119]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1641983232.067844]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1641983232.070791]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1641983232.073289]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1641983232.075933]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1641983

## Approach and grasp

In [124]:
below_goal_pose = sm.SE3([0,0,-0.05])@cobot.T_B_E_robot    
cobot.move_to_pose(below_goal_pose, err_gain = None, v_max = 0.01) 
force_z = np.array([])
T = int(100 * cobot.hz)
for t in trange(T):
    force_z = np.append(force_z,cobot.FT_F[2])
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    if abs(np.mean(force_z)-force_z[-1])>2:
        break
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1641983027.942873]: reached goal pose at step 395 / 10000


In [ ]:
#Enable Vacuum Gripper

## Dispose

In [125]:
## Go to pre_pose then dispose location, possible check if object is grasped
cobot.move_to_pose(pre_pose,err_gain = None, v_max = 0.03)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()
    #Keep Vacuum gripper enabled

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1641983031.750197]: reached goal pose at step 354 / 10000


In [126]:
#Enable Vacuum gripper
dispose_pose_quat = cu.homog2quat(dispose_pose)
dispose_pose_pos = cu.homog2pos(dispose_pose)

#position of end_configuration
goal.goal_pose.pose.position.x = dispose_pose_pos[0]
goal.goal_pose.pose.position.y = dispose_pose_pos[1]
goal.goal_pose.pose.position.z = dispose_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = dispose_pose_quat.x
goal.goal_pose.pose.orientation.y = dispose_pose_quat.y
goal.goal_pose.pose.orientation.z = dispose_pose_quat.z
goal.goal_pose.pose.orientation.w = dispose_pose_quat.w

client1.send_goal(goal)

In [116]:
#Disable vacuum gripper

#tool_controller.vacuum = True
cobot.move_to_pose(dispose_pose) 
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()